In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import time

# 전체 코드 실행 시작 시간 기록
overall_start_time = time.time()
print(f"시작 시간: {time.ctime(overall_start_time)}")

# 데이터 디렉토리 설정
data_directory = './data/result_wav/'  # 멜 스펙트로그램 데이터 디렉토리 경로

# 이미지 크기 및 하이퍼파라미터 설정
img_height, img_width = 64, 64  # 이미지 크기 설정 (64x64)
num_classes = 9  # 클래스 수 설정 (babycry, boil, carhorn 등 총 9개 클래스)
batch_size = 32  # 배치 크기 설정

# 데이터셋 준비
X_data = []  # 이미지 데이터 배열
y_data = []  # 레이블 배열

# 데이터 디렉토리에서 이미지 파일을 불러와서 리스트에 추가
for filename in os.listdir(data_directory):
    if filename.endswith('.png'):  # PNG 파일만 처리
        # 클래스 번호 추출 (파일명에서 첫 번째 요소를 추출)
        class_number = int(filename.split('_')[0])
        # 이미지 파일 경로 설정
        img_path = os.path.join(data_directory, filename)
        # 이미지 불러오기 및 전처리
        img = load_img(img_path, target_size=(img_height, img_width))  # 이미지 로드 및 크기 조정
        img_array = img_to_array(img) / 255.0  # 이미지를 배열로 변환 및 정규화
        X_data.append(img_array)  # 이미지 배열을 X_data 리스트에 추가
        y_data.append(class_number)  # 클래스 번호를 y_data 리스트에 추가

# 리스트를 numpy 배열로 변환
X_data = np.array(X_data)
y_data = np.array(y_data)

# 데이터셋을 학습 세트와 테스트 세트로 나누기
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=42)

# SVM 모델 생성
svm_model = SVC(kernel='linear', probability=True)

# 배치 단위로 SVM 모델 학습
num_batches = len(X_train) // batch_size

for i in range(num_batches):
    X_batch = X_train[i * batch_size:(i + 1) * batch_size]
    y_batch = y_train[i * batch_size:(i + 1) * batch_size]
    X_batch_flatten = X_batch.reshape(X_batch.shape[0], -1)
    svm_model.fit(X_batch_flatten, y_batch)

# 남은 데이터 처리
if len(X_train) % batch_size != 0:
    X_batch = X_train[num_batches * batch_size:]
    y_batch = y_train[num_batches * batch_size:]
    X_batch_flatten = X_batch.reshape(X_batch.shape[0], -1)
    svm_model.fit(X_batch_flatten, y_batch)

# 모델 평가
X_test_flatten = X_test.reshape(X_test.shape[0], -1)

y_pred = svm_model.predict(X_test_flatten)
test_acc = svm_model.score(X_test_flatten, y_test)
print(f"Test Accuracy: {test_acc}")

# 성능 보고서 및 혼동 행렬 출력
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=[str(i) for i in range(num_classes)]))  # 분류 보고서 출력
print("Confusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))  # 혼동 행렬 출력

# 전체 코드 실행 종료 시간 기록
overall_end_time = time.time()
print(f"종료 시간: {time.ctime(overall_end_time)}")
print(f"소요 시간: {overall_end_time - overall_start_time:.2f}초")


시작 시간: Fri Jul 26 21:51:16 2024
Test Accuracy: 0.4456089627755692
Classification Report:

              precision    recall  f1-score   support

           0       0.85      0.10      0.18       164
           1       0.58      0.55      0.57       241
           2       0.67      0.01      0.02       225
           3       0.38      0.67      0.48       424
           4       0.00      0.00      0.00       105
           5       0.00      0.00      0.00       162
           6       0.50      0.72      0.59       592
           7       0.41      0.53      0.46       701
           8       0.00      0.00      0.00       153

    accuracy                           0.45      2767
   macro avg       0.38      0.29      0.26      2767
weighted avg       0.42      0.45      0.38      2767

Confusion Matrix:

[[ 17   0   0  30   0   0  69  48   0]
 [  0 133   0  13   0   0  30  65   0]
 [  1  25   2  63   0   0  49  85   0]
 [  0   1   1 283   0   0  89  50   0]
 [  0   0   0  78   0   0  14 

C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\USER\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
